In [ ]:
import pickle

import cv2 as cv
import mediapipe as mp
import numpy as np

In [ ]:
model_dict = pickle.load(open('./model_abc_LSM.p', 'rb'))
model = model_dict['model']

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [ ]:
class_rgb = (('a (R)', (255, 0, 0)),      ('a (L)', (255, 0, 0)),
             ('b (R)', (235, 235, 0)),    ('b (L)', (235, 235, 0)),
             ('c (R)', (0, 200, 255)),    ('c (L)', (0, 200, 255)),
             ('d (R)', (170, 0, 255)),    ('d (L)', (170, 0, 255)),
             ('e (R)', (255, 127, 0)),    ('e (L)', (255, 127, 0)),
             ('f (R)', (191, 255, 0)),    ('f (L)', (191, 255, 0)),
             ('g (R)', (0, 149, 255)),    ('g (L)', (0, 149, 255)),
             ('h (R)', (255, 0, 170)),    ('h (L)', (255, 0, 170)),
             ('i (R)', (255, 212, 0)),    ('i (L)', (255, 212, 0)),
             ('l (R)', (106, 255, 0)),    ('l (L)', (106, 255, 0)),
             ('m (R)', (0, 64, 255)),     ('m (L)', (0, 64, 255)),
             ('n (R)', (237, 185, 185)),  ('n (L)', (237, 185, 185)),
             ('o (R)', (100, 100, 255)),  ('o (L)', (100, 100, 255)),
             ('p (R)', (200, 200, 185)),  ('p (L)', (200, 200, 185)),
             ('r (R)', (220, 185, 237)),  ('r (L)', (220, 185, 237)),
             ('s (R)', (165, 217, 204)),  ('s (L)', (165, 217, 204)),
             ('t (R)', (143, 35, 35)),    ('t (L)', (143, 35, 35)),
             ('u (R)', (35, 98, 143)),    ('u (L)', (35, 98, 143)),
             ('v (R)', (143, 106, 35)),   ('v (L)', (143, 106, 35)),
             ('w (R)', (107, 35, 143)),   ('w (L)', (107, 35, 143)),
             ('y (R)', (79, 143, 35)),    ('y (L)', (79, 143, 35)))

In [ ]:
cap = cv.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    #frame = cv.flip(frame, 1)
    H, W, _ = frame.shape

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,                                                    # image to draw
                hand_landmarks,                                           # model output
                mp_hands.HAND_CONNECTIONS)                                # Draw connections (default None)      connections
                #mp_drawing_styles.get_default_hand_landmarks_style()     # Colour points (default RED)          landmark_drawing_spec
                #mp_drawing_styles.get_default_hand_connections_style()   # Colour connections (default WHITE)   connection_drawing_spec
                #False                                                    # Remove points                        is_drawing_landmarks

        for hand_landmarks in results.multi_hand_landmarks:
            
            data_aux = []
            x_ = []
            y_ = []
            
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            min_x = min(x_)
            min_y = min(y_)
            max_x = max(x_)
            max_y = max(y_)
            
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append((x - min_x)/(max_x - min_x))
                data_aux.append((y - min_y)/(max_y - min_y))

            x1 = int(min(x_) * W) - 0
            y1 = int(min(y_) * H) - 0

            x2 = int(max(x_) * W) - 0
            y2 = int(max(y_) * H) - 0
            
            probabilities = model.predict_proba([np.asarray(data_aux)]) # Probabilites of each class
            n_prediction = np.argmax(probabilities)                     # Index of most probable class
            
            predicted_character, color = class_rgb[n_prediction]
            prediction_proba = str(np.max(probabilities))

            cv.rectangle(frame, (x1, y1), (x2, y1 - 45), color, -1)
            cv.rectangle(frame, (x1, y1), (x2, y2), color, 4)
            
            cv.putText(frame, 
                       predicted_character + ": " + prediction_proba, 
                       (x1 + 10, y1 - 10), 
                       cv.FONT_HERSHEY_COMPLEX,
                       0.85, 
                       (255, 255, 255), 
                       1, 
                       cv.LINE_AA)

    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()